In [2]:
import sys
import random, os
import psycopg2 as pg2 # Thư viện để kết nối DB
import pandas as pd
from faker import Faker # Thư viện để tạo seed data
from datetime import date
from dotenv import load_dotenv
sys.executable
print("Test Ok")

Test Ok


In [5]:
load_dotenv()
database=os.getenv("POSTGRES_DB")
user=os.getenv("POSTGRES_USER")
password=os.getenv("POSTGRES_PASSWORD")
host=os.getenv("POSTGRES_HOST")
port=os.getenv("POSTGRES_PORT")

# kết nối DB
conn = pg2.connect(database=database, user=user, password=password, host=host, port=port) 
cur = conn.cursor() # tạo con trỏ vào SQL

# kiểm tra kết nối với DB
try:
    cur.execute("SELECT 1")
    print("Kết nối DB thành công (Zora_Tasty)")
except:
    print("Lỗi kết nối DB! Kiểm tra lại!")
    exit()

Kết nối DB thành công (Zora_Tasty)


In [ ]:
seed = Faker('vi_VN') # tạo biến từ thư viện Faker lấy tên, sđt, địa chỉ bằng tiếng Việt

# lấy data từ csv các tên việt nam
names_df = pd.read_csv('../seeds/vietnamese_names_1000.csv')

# tạo hàm random tên VN
def random_vietnamese_names():
    row = names_df.sample(1).iloc[0]
    # sample(1): lấy ngẫu nhiên 1 hàng trong DataFrame (dạng bảng)
    # iloc[0]: lấy hàng đầu tiên (và duy nhất) trong kết quả đó, trả về kiểu Series (giống dict)
    
    return f"{row['full_name']}"
    # return f"{row['last_name']} {row['middle_name']} {row['first_name']}"
    # row lúc này là 1 dạng dict, với key là last_name, middle_name và first_name
    # nên ta gọi đúng key để trả về giá trị trong key đó (tên cụ thể của mỗi cột), và gộp lại thành tên hoàn chỉnh

# tạo hàm random phone (theo chuẩn các số đầu của VN)
def random_phone():
    prefix = random.choice(["032", "033", "034", "035", "036", "037", "038", "039", "070", "079", "081", "090", "091", "098"]) # random tiền tố
    suffix = ''.join(random.choices('0123456789', k=7)) # random hậu tố, lấy 7 số (k=7) từ các số từ 0-9
    return prefix + suffix

def get_age():
    age = random.randint(18, 90)
    return age

# xác suất có mail theo độ tuổi, nhỏ hơn 30t là 90%, nhỏ hơn 50t là 60%, trên 50 là 20%
def rate_email_by_age(age):
    if age < 30:
        return 0.9
    elif age < 50:
        return 0.6
    else:
        return 0.2

In [48]:
# test function
print(random_vietnamese_names())
print(random_phone())
test_age = get_age()
print(test_age)
print(rate_email_by_age(test_age))

Tô Anh Khánh
0904356566
23
0.9


In [49]:
# tạo các biến random thông tin và test
for i in range(20):
    name = random_vietnamese_names()
    phone = random_phone()
    birthdate = seed.date_of_birth(minimum_age=18, maximum_age=90)
    loyalty_points = 0
    #email = seed.email() if random.choice([True, False]) else None # random nhưng có lựa chọn có hay ko có mail (có thể rỗng)
    # random.choice([True, False]) chỉ trả về ngẫu nhiên True/False — nó không tự gán giá trị.
    # Còn if ... else ... xác định giá trị thực tế sẽ gán (email hoặc None).

    age = date.today().year - birthdate.year
    change = rate_email_by_age(age)
    email = seed.email() if random.random() < change else None
    
    print(' | '.join(map(str, [name, phone, birthdate, age, rate_email_by_age(age), email, loyalty_points]))) # map str toàn bộ

Ngô Hữu Quân | 0362636334 | 1965-05-06 | 60 | 0.2 | None | 0
Chu Thị Sơn | 0985170074 | 1970-08-28 | 55 | 0.2 | None | 0
Trương Thị Bình | 0810356919 | 1978-05-27 | 47 | 0.6 | None | 0
Chu Đức Lan | 0798396470 | 1944-10-08 | 81 | 0.2 | None | 0
Nguyễn Quang Châu | 0356111488 | 1958-08-09 | 67 | 0.2 | johntran@example.net | 0
Trần Bảo Nhi | 0346442556 | 1951-02-18 | 74 | 0.2 | None | 0
Trần Thanh Trung | 0388250421 | 1970-07-26 | 55 | 0.2 | None | 0
Phan Khánh Hạnh | 0797020711 | 1949-04-14 | 76 | 0.2 | None | 0
Nguyễn Phương Thịnh | 0819725613 | 1949-04-23 | 76 | 0.2 | None | 0
Phạm Anh Trung | 0901272109 | 1964-01-15 | 61 | 0.2 | None | 0
Ngô Hải Linh | 0982370148 | 1982-11-10 | 43 | 0.6 | lejohn@example.net | 0
Dương Anh Dũng | 0908129532 | 1949-07-03 | 76 | 0.2 | None | 0
Phạm Văn Vy | 0372124070 | 1955-03-26 | 70 | 0.2 | None | 0
Tô Hữu Nhi | 0361417106 | 1990-06-22 | 35 | 0.6 | johnle@example.org | 0
Trần Gia Khánh | 0326382726 | 1981-02-03 | 44 | 0.6 | jane04@example.net | 0
Ngô 

In [ ]:
# đưa dữ liệu đã random vào DB (bảng tạm hệ thống)
for i in range(20):
    name = random_vietnamese_names()
    phone = random_phone()
    birthdate = seed.date_of_birth(minimum_age=18, maximum_age=90)
    age = date.today().year - birthdate.year
    change = rate_email_by_age(age)
    email = seed.email() if random.random() < change else None
    cur.execute("""
        INSERT INTO Customers (customer_name, phone, email, birthdate, loyalty_points)
        VALUES (%s, %s, %s, %s, %s)
        RETURNING customer_id
    """, (name, phone, email, birthdate, loyalty_points))
    # %s ở đây là placeholder (vị trí chờ) cho các giá trị thực tế (để gán các biến random ở trên)

    customer_ids = []
    customer_id = cur.fetchone()[0]  # lấy id vừa tạo (vị trí 0 là id)
    customer_ids.append(customer_id)
    print(' | '.join(map(str, [name, phone, birthdate, age, rate_email_by_age(age), email, loyalty_points])))

Phạm Nguyễn Quốc Huy Sơn | 0385495074 | 1983-06-07 | 42 | 0.6 | None | 0
Võ Hoàng Sơn | 0986972997 | 1964-03-20 | 61 | 0.2 | None | 0
Dương Bảo Ngọc Vy | 0381526229 | 1961-08-26 | 64 | 0.2 | None | 0
Trần Anh Linh | 0705559835 | 1991-11-05 | 34 | 0.6 | dpham@example.com | 0
Huỳnh Phương Dũng | 0360215942 | 1969-09-23 | 56 | 0.2 | None | 0
Võ Hoàng Sơn | 0384592817 | 2001-11-05 | 24 | 0.9 | mtran@example.com | 0
Phạm Ngọc Sơn | 0700432107 | 1982-01-05 | 43 | 0.6 | None | 0
Đỗ Bảo Ngọc Khánh | 0817674364 | 1981-11-01 | 44 | 0.6 | None | 0
Tạ Hoàng Thịnh | 0376331786 | 1953-10-20 | 72 | 0.2 | None | 0
Bùi Ngọc Bích Thịnh | 0370427176 | 1955-12-18 | 70 | 0.2 | None | 0
Lê Anh Thư | 0983315034 | 1999-02-13 | 26 | 0.9 | None | 0
Tạ Ngọc Ngọc | 0382190268 | 1955-12-03 | 70 | 0.2 | None | 0
Nguyễn Ngọc Bích Quân | 0339130495 | 1980-09-03 | 45 | 0.6 | None | 0
Võ Ngọc Bảo Ngọc Bình | 0811706259 | 1972-01-05 | 53 | 0.2 | fmai@example.net | 0
Vũ Ngọc Diệu Hằng Trâm | 0900485433 | 1982-03-26 | 43 

In [ ]:
conn.rollback() # xóa bảng lưu tạm để chạy lại

In [ ]:
cur.execute('TRUNCATE TABLE Customers RESTART IDENTITY CASCADE') # xóa data trên DB và rs Index về 1
print(f"ĐÃ XÓa!")

ĐÃ XÓa!


In [ ]:
conn.commit() # lưu thực tế vào psql
print(f"ĐÃ TẠO!")

ĐÃ TẠO 1 !


In [ ]:
cur.execute('SELECT * FROM Customers') # câu lệnh nằm chờ trong bộ đệm kết quả
rows = cur.fetchall() # lấy dữ liệu bằng fetch
if len(rows) > 0:
    for i in rows:
        print(i)
else:
    print('Data rỗng')

(1, 'Phạm Nguyễn Quốc Huy Sơn', '0385495074', None, datetime.date(1983, 6, 7), 0, datetime.datetime(2025, 11, 6, 4, 56, 26, 619482), None)
(2, 'Võ Hoàng Sơn', '0986972997', None, datetime.date(1964, 3, 20), 0, datetime.datetime(2025, 11, 6, 4, 56, 26, 619482), None)
(3, 'Dương Bảo Ngọc Vy', '0381526229', None, datetime.date(1961, 8, 26), 0, datetime.datetime(2025, 11, 6, 4, 56, 26, 619482), None)
(4, 'Trần Anh Linh', '0705559835', 'dpham@example.com', datetime.date(1991, 11, 5), 0, datetime.datetime(2025, 11, 6, 4, 56, 26, 619482), None)
(5, 'Huỳnh Phương Dũng', '0360215942', None, datetime.date(1969, 9, 23), 0, datetime.datetime(2025, 11, 6, 4, 56, 26, 619482), None)
(6, 'Võ Hoàng Sơn', '0384592817', 'mtran@example.com', datetime.date(2001, 11, 5), 0, datetime.datetime(2025, 11, 6, 4, 56, 26, 619482), None)
(7, 'Phạm Ngọc Sơn', '0700432107', None, datetime.date(1982, 1, 5), 0, datetime.datetime(2025, 11, 6, 4, 56, 26, 619482), None)
(8, 'Đỗ Bảo Ngọc Khánh', '0817674364', None, datetim